# OBJECT DETECTION

## Template Matching

Template Matching is a method for searching and finding the location of a template image in a larger image

In [26]:
import cv2 
import numpy as np

img = cv2.imread('card.png',0)
cv2.imshow("Original", img)

template = cv2.imread('card_temp.png',0)
w, h = template.shape[::-1]

# evaluates cv2.method_name dynamically (when program runs) 
method = eval("cv2.TM_CCOEFF")

# matchTemplate(image, template, method)
res = cv2.matchTemplate(img,template,method)

# find min and max locations and values
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

# find bottom right of position of template
# max_loc gives us top left position of template as (x,y)
bottom_right = (max_loc[0] + w, max_loc[1] + h)

# put rectangel of this position
cv2.rectangle(img,top_left, bottom_right, 0, 2)

cv2.imshow("Template Matching", img)

cv2.waitKey()
cv2.destroyAllWindows()

In [25]:
import cv2 
import numpy as np

img = cv2.imread('card.png')
cv2.imshow("Original", img)

gray = cv.cvtColor(img, cv2.COLOR_BGR2GRAY)

template = cv2.imread('card_temp.png',0)
w, h = template.shape[::-1]

res = cv.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)

threshold = 0.6
# if match result greater than threshold.
loc = np.where( res >= threshold)

# put rectangle to all locations
for pt in zip(*loc[::-1]):
    cv.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0,0,255), 1)

cv2.imshow("Template Matching", img)

cv2.waitKey()
cv2.destroyAllWindows()

## Hough Transform

The Hough Transform is a popular technique to detect any shape, if you can represent that shape in a mathematical form. It can detect the shape even if it is broken or distorted a little bit. There is 2 image which are include bumb and scratch of car

## Hough Line Detection

In [114]:
import cv2
import numpy as np

img = cv2.imread('scratch.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Canny(image, threshold1, threshold2, order of kernel)
edges = cv2.Canny(gray,5,85,apertureSize = 3)

# HoughLines(image,rho, theta, threshold)
lines = cv2.HoughLines(edges,1,np.pi/180,100)

length = 0
loc = []

# if you want to understand the math below check
# https://github.com/adityaintwala/Hough-Line-Detection
for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    
    # find length of line
    cur_len = ((x2-x1)**2 + (y2-y1)**2)**0.5
    # find max length
    if length < cur_len:
        length = cur_len 
        loc
        loc = [(x1,y1),(x2,y2)]

cv.line(img,loc[0],loc[1],(0,0,255),1)

cv2.imshow("Detected Lines", img)

cv2.waitKey()
cv2.destroyAllWindows()

## Probabilistic Hough Transform

In [ ]:
import cv2
import numpy as np

img = cv2.imread('scratch.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,5,85,apertureSize = 3)

# HoughLinesP(image,rho,theta,threshold,minLineLength,maxLineGap)
lines = cv.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=10)
for line in lines:
    x1,y1,x2,y2 = line[0]
    cv.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow("Detected Lines", img)

cv2.waitKey()
cv2.destroyAllWindows()

## Hough Circle Detection

In [116]:
import cv2
import numpy as np

img = cv2.imread('bumb.png',0)
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
# HoughCircles(image, method, Resolution of the accumulator array, Minimum distance between the center,
# gradşent value, threshold value of method, min radius, max radius)
circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,param1=50,param2=30,minRadius=0,maxRadius=0)
circles = np.uint16(np.around(circles))

for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    
cv2.imshow('detected circles',cimg)
cv2.waitKey()
cv2.destroyAllWindows()

## Contours

If we want to detect contour, we need to clear binary image.

In [4]:
import cv2
import numpy as np

img = cv2.imread('coins.png')

cv2.imshow('Original',img)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# you can find clear output by change values
gray_blur = cv2.GaussianBlur(gray, (15, 15), 0)
thresh = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 5, 1)

kernel = np.ones((7,7), np.uint8)
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE,kernel, iterations=3)

cv2.imshow('closing',closing)

# now we can find contours
# findContours(image, method_1, method_2)
contours, hierarchy = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# let's inspect contours
print(len(contours), "contour found")
for contour in contours:
    # you can find contour area by using contourArea()
    area = cv2.contourArea(contour)
    print("contour area: ", area)
    
    # you can find contour perimeter by using arcLength()
    perimeter = cv2.arcLength(contour,True)
    print("contour perimeter: ", perimeter)
    
    # you can create rectangle which includes contour
    x,y,w,h = cv2.boundingRect(contour)
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0),1)
    
    # you can fit contour to ellipse
    ellipse = cv2.fitEllipse(contour)
    cv2.ellipse(img, ellipse, (0,255,0), 2)
    
    
    
cv2.imshow("contours", img)  


cv2.waitKey()
cv2.destroyAllWindows()

6 contour found
contour area:  18005.5
contour perimeter:  520.9188272953033
contour area:  18394.5
contour perimeter:  527.5462448596954
contour area:  18987.0
contour perimeter:  537.5046153068542
contour area:  22116.5
contour perimeter:  571.4873690605164
contour area:  18297.5
contour perimeter:  534.0315253734589
contour area:  19738.0
contour perimeter:  547.7888844013214


## Corner Detection

In [6]:
import cv2
import numpy as np

img = cv2.imread('figures.jpg',)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# we need a float numpy array
gray2 = np.float32(gray)

# cornerHarris(image, block_size, aperture_size,k)
dst = cv2.cornerHarris(gray2,2,3,0.04)
cv2.imshow('First',dst)

dst2 = cv2.dilate(dst,None)
cv2.imshow('Second',dst2)

# eliminate corners
img[dst>0.3*dst.max()/4]=[0,0,255]
cv2.imshow('last',img)

cv2.waitKey()
cv2.destroyAllWindows()

## Count Circles and Ellipses

In [62]:
import cv2
import numpy as np

def nothing(x):
    pass

image2 = cv2.imread('figures.jpg')

cv2.imshow('Original',image2)


cv2.namedWindow("Trackbars")
# L: lower U: Upper
# you can find perfect mask for hsv space
cv2.createTrackbar("L-H", "Trackbars", 0,180, nothing)
cv2.createTrackbar("L-S", "Trackbars", 0,255, nothing)
cv2.createTrackbar("L-V", "Trackbars", 0,255, nothing)
cv2.createTrackbar("U-H", "Trackbars", 180,180, nothing)
cv2.createTrackbar("U-S", "Trackbars", 255,255, nothing)
cv2.createTrackbar("U-V", "Trackbars", 255,255, nothing)

font = cv2.FONT_HERSHEY_COMPLEX
line = cv2.LINE_AA
    
while True:
    image = cv2.imread('figures.jpg')
    hsv = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
    # get values from trackbar
    l_h = cv2.getTrackbarPos("L-H","Trackbars")
    l_s = cv2.getTrackbarPos("L-S","Trackbars")
    l_v = cv2.getTrackbarPos("L-V", "Trackbars")
    u_h = cv2.getTrackbarPos("U-H", "Trackbars")
    u_s = cv2.getTrackbarPos("U-S", "Trackbars")
    u_v = cv2.getTrackbarPos("U-V", "Trackbars") 
    
    black_upper = np.array([u_h, u_s, u_v])
    black_lower = np.array([l_h, l_s, l_v])    
    mask = cv2.inRange(hsv,black_lower,black_upper)
    
    blur = cv2.GaussianBlur(mask, (5, 5), 0)

    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(blur, kernel)
    mask = cv2.dilate(mask, kernel)
    cv2.imshow('last',mask)
    
    triangles = 0
    circles = 0
    squares = 0
    rectangle = 0
    
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 500 or area > 5500:
            continue
        [ref_x, ref_y, ref_width, ref_height] = cv2.boundingRect(contour)
        ref_approx = cv2.approxPolyDP(contour, 0.04 * cv2.arcLength(contour, True), True)
        cv2.drawContours(image, [ref_approx], 0, (0, 0, 255), 3)
        
        if(len(ref_approx) == 3):
            triangles +=1
            cv2.putText(image, "triangle:"+str(triangles), (ref_x, ref_y), font, 0.4, (255, 0, 0), 1, line)
        elif(len(ref_approx) == 4):
            ar = ref_width / float(ref_height)
            if(ar >= 0.96 and ar <=1.05):
                squares +=1
                cv2.putText(image, "square:"+str(squares), (ref_x, ref_y), font, 0.4, (255, 0, 0), 1, line)
            else:
                rectangle +=1
                cv2.putText(image, "rectangle:"+str(rectangle), (ref_x, ref_y), font, 0.4, (255, 0, 0), 1, line) 
        else:
            circles += 1
            cv2.putText(image, "circles:"+str(circles), (ref_x, ref_y), font, 0.4, (255, 0, 0), 1, line)   
            
    
    cv2.imshow('final',image)
    # if user press q break loop
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
cv2.destroyAllWindows()

Note: If you couldn't find the mask value try for this values below:
L-H:96
L-S:111
L-V:0
U-H:180
U-S:255
U-V:255